# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from z3 import *
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("16-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'#############################################################################################################################################\n#.....#.......#.............#.........#.......................#.......#...#...............#.....#.#.................................#.#....E#\n#.#####.###.###.#######.###.#.#.#######.#.###.#####.###.#######.#.###.###.#.#####.#####.###.#.#.#.#.#####.###.#.###.#######.###.#.#.#.#.#.#.#\n#.......#...#...#.....#.#.#.#.#.#...#...#.....#...#.#...#.......#...#.#...#...#...#.#...#...............#...#...#.#.......#.#...#.#.#...#.#.#\n#########.#.#.###.#####.#.#.#.#.#.#.#.#########.#.#.###.#.#########.#.#.#.#.###.#.#.#.###.###.#####.###.###.#.#.#.###.###.#.#.###.#.#.###.#.#\n#.......#.#.#.#...#...#.#.#.#.#...#.....#.......#.#...#.#.#...#...#.#...#.#.#...#.....#...#.#...#...#.....#.#.....#.....#.#...#...#.#.#...#.#\n#.#####.#.###.#.#.#.#.#.#.#.###########.#######.#.###.#.#.###.#.#.#.#.#####.#.###.###.#.###.###.#.###.#.###.#.#.#.#.#.#.#.#.#.#.###.###.#####

In [4]:
test_data_raw = r"""###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""

test_data_raw2 = r"""#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################"""

Robot = namedtuple("Robot", ["px", "py","vx", "vy"])

def preprocess_data (data):


    rows = [list(row) for row in data.split("\n")]
    return np.array(rows, dtype='U10')



test_data = preprocess_data(test_data_raw)
test_data2 = preprocess_data(test_data_raw2)
display(test_data)


array([['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.',
        'E', '#'],
       ['#', '.', '#', '.', '#', '#', '#', '.', '#', '.', '#', '#', '#',
        '.', '#'],
       ['#', '.', '.', '.', '.', '.', '#', '.', '#', '.', '.', '.', '#',
        '.', '#'],
       ['#', '.', '#', '#', '#', '.', '#', '#', '#', '#', '#', '.', '#',
        '.', '#'],
       ['#', '.', '#', '.', '#', '.', '.', '.', '.', '.', '.', '.', '#',
        '.', '#'],
       ['#', '.', '#', '.', '#', '#', '#', '#', '#', '.', '#', '#', '#',
        '.', '#'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#',
        '.', '#'],
       ['#', '#', '#', '.', '#', '.', '#', '#', '#', '#', '#', '.', '#',
        '.', '#'],
       ['#', '.', '.', '.', '#', '.', '.', '.', '.', '.', '#', '.', '#',
        '.', '#'],
       ['#', '.', '#', '.', '#', '.', '#', '#', '#', '.', '#', '.', '#',
       

In [5]:
full_data = preprocess_data(data_raw)
full_data

array([['#', '#', '#', ..., '#', '#', '#'],
       ['#', '.', '.', ..., '.', 'E', '#'],
       ['#', '.', '#', ..., '#', '.', '#'],
       ...,
       ['#', '.', '#', ..., '#', '.', '#'],
       ['#', 'S', '.', ..., '.', '.', '#'],
       ['#', '#', '#', ..., '#', '#', '#']], dtype='<U10')

In [33]:

def solution (data, verbose=False):
    field = data
    

    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str

    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1])

    def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))
    
    def get_neightbors (array, pos):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        symbol = ["v", "^", ">", "<"]
        next_nodes = [(tuple_add(pos, direction), symbol) for direction, symbol in zip(directions, symbol) 
                                                                if isValid(array.shape, tuple_add(pos, direction)) 
                                                                   and array[tuple_add(pos, direction)] in ['.', "E"] ]
        return next_nodes
    
    # print(display_field(field))
    start = (tuple(np.argwhere(field == 'S')[0]), '>')
    end_nodes = [(tuple(np.argwhere(field == 'E')[0]), dir) for dir in  ["<", ">", "v", "^"]]

    allowed_turns = {"<": ["v", "^"], ">": ["v", "^"], "v":["<", ">"], "^":["<", ">"]}

    visited = []
    queue = [start]
    costs = defaultdict(lambda: float('inf'))
    costs[start] = 0

    while queue:
        # find smallest
        current_node = queue[0]
        # print(current_node)
        lowest_cost = costs[current_node]
        if len(queue) > 1:
            for node in queue[1:]:
                if costs[node] < lowest_cost:
                    current_node = node
                    lowest_cost = costs[node]

        queue.remove(current_node)
        visited.append(current_node)

        nbs = get_neightbors(field, current_node[0])
        # print("Neighbors", nbs)
        for nb in nbs:
            if nb not in visited:
                if current_node[1] == nb[1] and costs[nb] > costs[current_node] +1:
                    costs[nb] = costs[current_node] +1
                    queue.append(nb)
                elif nb[1] in allowed_turns[current_node[1]] and costs[nb] > costs[current_node] +1000:
                    costs[nb] = costs[current_node] +1000+1
                    queue.append(nb)

    end_nodes_costs = [costs[node] for node in end_nodes]

    # display(costs)
    # print(end_nodes)
    return min(end_nodes_costs)



sol = solution(test_data, verbose=True)
print(sol)

sol = solution(test_data2, verbose=True)
print(sol)



7036
11048


In [22]:
sol = solution(full_data)
print(sol)

134588


# Part 2

In [28]:

def solution2 (data, verbose=False):
    field = data
    

    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str

    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1])

    def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))
    
    def get_neightbors (array, pos):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        symbol = ["v", "^", ">", "<"]
        next_nodes = [(tuple_add(pos, direction), symbol) for direction, symbol in zip(directions, symbol) 
                                                                if isValid(array.shape, tuple_add(pos, direction)) 
                                                                   and array[tuple_add(pos, direction)] in ['.', "E"] ]
        return next_nodes
    
    # print(display_field(field))
    start = (tuple(np.argwhere(field == 'S')[0]), '>')
    end_nodes = [(tuple(np.argwhere(field == 'E')[0]), dir) for dir in  ["<", ">", "v", "^"]]

    allowed_turns = {"<": ["v", "^"], ">": ["v", "^"], "v":["<", ">"], "^":["<", ">"]}

    visited = set()
    queue = [start]
    costs = defaultdict(lambda: float('inf'))
    costs[start] = 0
    predecessor = defaultdict(list)
    print("Forward search...")

    max_steps = (len(np.argwhere(field=='.').tolist()) +2)*4
    step = 1
    while queue:
        print("\r"+ f"Step {step} of at most {max_steps} steps. {max_steps/step:.0%}", end="")    
        step+= 1

        # find smallest
        current_node = queue[0]
        # print(current_node)
        lowest_cost = costs[current_node]
        if len(queue) > 1:
            for node in queue[1:]:
                if costs[node] < lowest_cost:
                    current_node = node
                    lowest_cost = costs[node]

        queue.remove(current_node)
        visited.add(current_node)

        nbs = get_neightbors(field, current_node[0])
        # if we are on an end node we stop 
        if current_node in end_nodes:
            nbs =[]
        # print("Neighbors", nbs)
        for nb in nbs:
            
            if current_node[1] == nb[1] and costs[nb] >= costs[current_node] +1:

                if costs[nb] == costs[current_node] +1:
                    predecessor[nb].append(current_node)
                else:
                    predecessor[nb] = [current_node]

                if nb not in visited and nb not in queue:
                    costs[nb] = costs[current_node] +1
                    queue.append(nb)

            elif nb[1] in allowed_turns[current_node[1]] and costs[nb] >= costs[current_node] +1000+1:
                if costs[nb] == costs[current_node] +1000+1:
                    predecessor[nb].append(current_node)
                else:
                    predecessor[nb] = [current_node]

                if nb not in visited and nb not in queue:
                    costs[nb] = costs[current_node] +1000+1
                    queue.append(nb)
            
    
    print("\r"+ f"Done"  + " "*50)  
    print("Backward Search...")
    end_nodes_costs = [costs[node] for node in end_nodes]

    path_field = field.copy()
    pred = [node for node in end_nodes if costs[node] == min(end_nodes_costs)]
    steps = 0
    visited = set()
    max_steps = len(np.argwhere(field=='.').tolist()) +2
    while pred:
        steps +=1
        if steps % 100 == 0:
            print("\r"+ f"Step {steps} of at most {max_steps} steps", end="")
        current_node = pred.pop()
        path_field[current_node[0]] = 'X'
        # print(display_field(path_field))
        # print(current_node)
        nodes_to_add = [node for node in predecessor[current_node] if node not in visited]
        pred = pred + nodes_to_add

        path_field[current_node[0]] = 'O'
        visited.add(current_node)

    print("\r"+ f"Done" + " "*50)  
    # print(display_field(path_field))

    # display(costs)
    # print(end_nodes)
    return len(np.argwhere(path_field=='O').tolist())



sol = solution2(test_data, verbose=True)
print("Solution 45",sol)

sol = solution2(test_data2, verbose=True)
print("Solution 64", sol)

Forward search...
Done                                                  
Backward Search...
Done                                                  
Solution 45 45
Forward search...
Done                                                  
Backward Search...
Done                                                  
Solution 64 64


In [29]:
sol = solution2(full_data)
print("Solution", sol)

Forward search...
Done                                                  
Backward Search...
Done                                                  
Solution 631
